In [1]:
import sys
print("Python Version is: " + sys.version)



Python Version is: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [8]:
import time
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib #load model
import subprocess
from sklearn import feature_selection
#
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,RidgeClassifier,SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder,StandardScaler , MinMaxScaler, MaxAbsScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from lightgbm import LGBMClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix, classification_report,f1_score,accuracy_score 
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier

#imblen learn
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
 
# Get multiple outputs in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
 
# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x) #goes to two decimal places

In [9]:
import time
time_begin = time.time()

df = pd.read_csv("CleanedDF.csv") # data = pd.read_csv("census.csv")

print(f'Run time: {round(((time.time()-time_begin)/60), 3)} mins')

Run time: 0.001 mins


In [10]:
X = df
y = df['Churn']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                    stratify=y,
                                                    shuffle = True
                                                   )

In [12]:
remove = ['CustomerID','ZipCode','City', 'ChurnCategory','ChurnReason','Churn']

# numercical columns
num_feats = [ 
 'Age',
 'NumberofDependents',
 'Population',
 'NumberofReferrals',
 'TenureinMonths',
 'AvgMonthlyLongDistanceCharges',
 'AvgMonthlyGBDownload',
 'MonthlyCharge',
 'TotalCharges',
 'TotalRefunds',
 'TotalExtraDataCharges',
 'TotalLongDistanceCharges',
 'TotalRevenue'
]
# categorical columns
cat_feats = [ 
 'Gender',
 'Offer',
 'Married',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'InternetService',
 'InternetType',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtectionPlan',
 'PremiumTechSupport',
 'StreamingTV',
 'StreamingMovies',
 'StreamingMusic',
 'UnlimitedData',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
]

In [13]:
X_test['Churn'].value_counts()

Churn
0    1537
1     558
Name: count, dtype: int64

In [14]:
X_testcopy = X_test.copy()
X_testcopy.sample(2)

,CustomerID,Gender,Age,Married,NumberofDependents,City,ZipCode,Population,NumberofReferrals,TenureinMonths,Offer,PhoneService,AvgMonthlyLongDistanceCharges,MultipleLines,InternetService,InternetType,AvgMonthlyGBDownload,OnlineSecurity,OnlineBackup,DeviceProtectionPlan,PremiumTechSupport,StreamingTV,StreamingMovies,StreamingMusic,UnlimitedData,Contract,PaperlessBilling,PaymentMethod,MonthlyCharge,TotalCharges,TotalRefunds,TotalExtraDataCharges,TotalLongDistanceCharges,TotalRevenue,Churn,ChurnCategory,ChurnReason
5793,8218-FFJDS,0,21,1,0,Georgetown,95634,2723,6,72,NaN,1,15.35,Yes,1,DSL,41.00,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Two Year,1,Bank Withdrawal,86.40,6058.95,0.00,0,1105.20,7164.15,0,Unknown,Unknown
3160,4560-WQAQW,0,62,0,0,Mount Laguna,91948,81,0,59,Offer B,1,28.32,Yes,1,DSL,9.00,Yes,No,No,Yes,No,Yes,Yes,No,One Year,0,Bank Withdrawal,68.70,4070.95,0.00,150,1670.88,5891.83,0,Unknown,Unknown


In [15]:
X_test.drop(remove, axis = 1, inplace = True)
X_train.drop(remove, axis = 1, inplace = True)

In [16]:
#X_test.drop(['Churn'], axis = 1, inplace = True)
#X_train.drop(['Churn'], axis = 1, inplace = True)
#X_test.sample(2)

In [17]:
def get_pipeline(X, model): 

    numeric_pipeline = SimpleImputer(strategy='mean')
    categorical_pipeline = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_pipeline, num_feats),
            ('categorical', categorical_pipeline, cat_feats),
            ], remainder='passthrough'
    )

    bundled_pipeline = imbpipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('scaler', MinMaxScaler()),
        ('model', model)
    ])
    
    return bundled_pipeline

In [21]:
def select_model(X, y, pipeline=None):  
    classifiers = {}
    classifiers.update({"DummyClassifier": DummyClassifier(strategy='most_frequent')})
    classifiers.update({"XGBClassifier": XGBClassifier(use_label_encoder=False, 
                                                       eval_metric='logloss',
                                                       objective='binary:logistic',
                                                      )})
    classifiers.update({"LGBMClassifier": LGBMClassifier()})
    classifiers.update({"RandomForestClassifier": RandomForestClassifier()})
    classifiers.update({"DecisionTreeClassifier": DecisionTreeClassifier()})
    classifiers.update({"ExtraTreeClassifier": ExtraTreeClassifier()})
    #classifiers.update({"ExtraTreesClassifier": ExtraTreeClassifier()})    
    classifiers.update({"AdaBoostClassifier": AdaBoostClassifier()})
    classifiers.update({"KNeighborsClassifier": KNeighborsClassifier()})
    classifiers.update({"RidgeClassifier": RidgeClassifier()})
    classifiers.update({"SGDClassifier": SGDClassifier()})
    classifiers.update({"BaggingClassifier": BaggingClassifier()})
    classifiers.update({"BernoulliNB": BernoulliNB()})
    classifiers.update({"SVC": SVC()})
    classifiers.update({"CatBoostClassifier":CatBoostClassifier(silent=True)})
    
    # Stacking
    models = []

    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
    models.append(('BaggingClassifier', BaggingClassifier()))
    classifiers.update({"VotingClassifier (XGBClassifier, CatBoostClassifier, BaggingClassifier)": VotingClassifier(models)})

    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('LGBMClassifier', LGBMClassifier()))
    models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
    classifiers.update({"VotingClassifier (XGBClassifier, LGBMClassifier, CatBoostClassifier)": VotingClassifier(models)})
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('RandomForestClassifier', RandomForestClassifier()))
    models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
    classifiers.update({"VotingClassifier (XGBClassifier, RandomForestClassifier, DecisionTreeClassifier)": VotingClassifier(models)})

    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('AdaBoostClassifier', AdaBoostClassifier()))
    #models.append(('ExtraTreeClassifier', ExtraTreeClassifier()))
    classifiers.update({"VotingClassifier (XGBClassifier, AdaBoostClassifier)": VotingClassifier(models)})
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    #models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
    classifiers.update({"VotingClassifier (XGBClassifier)": VotingClassifier(models)})    
    
    df_models = pd.DataFrame(columns=['model', 'run_time', 'accuracy'])

    for key in classifiers:
        
        start_time = time.time()

        pipeline = get_pipeline(X_train, classifiers[key])
        
        cv = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')

        row = {'model': key,
               'run_time': format(round((time.time() - start_time)/60,2)),
               'accuracy': cv.mean(),
        }

        df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)
        
    df_models = df_models.sort_values(by='accuracy', ascending=False)
    return df_models

In [22]:
models = select_model(X_train, y_train)

[LightGBM] [Info] Number of positive: 2866, number of negative: 2866
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004442 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13011
[LightGBM] [Info] Number of data points in the train set: 5732, number of used features: 68
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 2867, number of negative: 2867
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004164 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12950
[LightGBM] [Info] Number of data points in the train set: 5734, number of used features: 68
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 2867, number of negative: 2867
[LightGBM] [Info] Auto-choosing col-wise multi-threading,

In [23]:
models.sort_values(by=['accuracy','run_time'], ascending=False)

,model,run_time,accuracy
13,CatBoostClassifier,1.13,0.85
15,"VotingClassifier (XGBClassifier, LGBMClassifie...",0.86,0.85
14,"VotingClassifier (XGBClassifier, CatBoostClass...",1.53,0.85
2,LGBMClassifier,0.05,0.84
17,"VotingClassifier (XGBClassifier, AdaBoostClass...",0.23,0.84
18,VotingClassifier (XGBClassifier),0.07,0.84
1,XGBClassifier,0.07,0.84
16,"VotingClassifier (XGBClassifier, RandomForestC...",0.4,0.83
3,RandomForestClassifier,0.3,0.83
6,AdaBoostClassifier,0.18,0.83


In [24]:
basemodel = XGBClassifier(use_label_encoder = False, eval_metric='logloss', objective='binary:logistic')

In [25]:
bundled_pipeline = get_pipeline(X_train, basemodel)

In [26]:
bundled_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric', SimpleImputer(),
                                                  ['Age', 'NumberofDependents',
                                                   'Population',
                                                   'NumberofReferrals',
                                                   'TenureinMonths',
                                                   'AvgMonthlyLongDistanceCharges',
                                                   'AvgMonthlyGBDownload',
                                                   'MonthlyCharge',
                                                   'TotalCharges',
                                                   'TotalRefunds',
                                                   'TotalExtraDataCharges',
                                                   'TotalLongDistanceCharges',
                                                   'Tota...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [27]:
basemodel_y_pred = bundled_pipeline.predict(X_test)

In [28]:
print(classification_report(y_test, basemodel_y_pred))
print(confusion_matrix(y_test, basemodel_y_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      1537
           1       0.72      0.66      0.69       558

    accuracy                           0.84      2095
   macro avg       0.80      0.78      0.79      2095
weighted avg       0.84      0.84      0.84      2095

[[1393  144]
 [ 188  370]]


### Runing Test/Train split through pipeline w/hyper-parameter tuning

In [29]:
time_begin = time.time() #starts timer

#Loan Model

model = XGBClassifier(
    use_label_encoder = False, eval_metric='logloss', objective='binary:logistic', learning_rate = 0.1,
                     n_estimators = 1000, max_depth = 9, min_child_weight = 1, gamma = 0.4, colsample_bytree = 0.8, 
                      subsample = 0.9, reg_alpha = 1, scale_pos_weight = 1)

model = get_pipeline(X_train,model)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

#predict target probabilities
test_prob = model.predict_proba(X_test)[:,1]

test_pred = np.where(test_prob > 0.45, 1, 0) #sets the probability threshhold and can be tweaked

#test set metrics
roc_auc_score(y_test, test_pred)
recall_score(y_test, test_pred)
confusion_matrix(y_test, test_pred)

print(classification_report(y_test,test_pred))

print(f'Run time: {round(((time.time()-time_begin)/60), 3)} mins')

# adding predictions and their probabilities to the original test Data frame
X_testcopy['predictions'] = test_pred
X_testcopy['pred_probabilities'] = test_prob

high_churn_list = X_testcopy[X_testcopy.pred_probabilities > 0.0].sort_values(by=['pred_probabilities'], ascending = False
                                                                             ).reset_index().drop(columns=['index'],axis=1)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric', SimpleImputer(),
                                                  ['Age', 'NumberofDependents',
                                                   'Population',
                                                   'NumberofReferrals',
                                                   'TenureinMonths',
                                                   'AvgMonthlyLongDistanceCharges',
                                                   'AvgMonthlyGBDownload',
                                                   'MonthlyCharge',
                                                   'TotalCharges',
                                                   'TotalRefunds',
                                                   'TotalExtraDataCharges',
                                                   'TotalLongDistanceCharges',
                                                   'Tota...
                               feature_types=None, gamma=0.4, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=9, max_leaves=None, min_child_weight=1,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=1000,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=None, ...))])

0.7943195677470659

0.6953405017921147

array([[1373,  164],
       [ 170,  388]], dtype=int64)

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1537
           1       0.70      0.70      0.70       558

    accuracy                           0.84      2095
   macro avg       0.80      0.79      0.80      2095
weighted avg       0.84      0.84      0.84      2095

Run time: 0.051 mins


In [30]:
high_churn_list.to_csv('high_churn_list_model.csv', index = False)